In [64]:
import requests
import os
import pandas as pd
import base64
from dotenv import load_dotenv
from datetime import datetime, timedelta
import time
from urllib.parse import parse_qs, urlparse
# Load environment variables from .env file
load_dotenv()

# Configuration using environment variables
GITHUB_TOKEN = os.getenv("GITHUB_API_KEY")  # Common name used for GitHub tokens
BASE_URL = "https://api.github.com"  # Base URL for GitHub API

# Create a session for reuse
session = requests.Session()
session.headers.update({
    "Authorization": f"Bearer {GITHUB_TOKEN}",  # Note: no space after "Authorization"
    "Accept": "application/vnd.github+json",    # Specify desired response format
    "X-GitHub-Api-Version": "2022-11-28"        # Specify API version
})


In [77]:
def test_api_connection():
    """Test the GitHub API connection with current authentication."""
    response = session.get(f"{BASE_URL}/user")
    if response.status_code == 200:
        user_data = response.json()
        print(f"Successfully connected as: {user_data['login']}")
        return user_data
    else:
        print(f"Connection failed: {response.status_code}")
        print(response.json())
        return None

# Function to get repositories as a pandas DataFrame
def get_my_repositories(include_forks=False):
    """Get repositories for the authenticated user or a specific user as a pandas DataFrame."""
    response = session.get(f"{BASE_URL}/user/repos", params={'per_page': 100})

    if response.status_code == 200:
        repos = response.json()

        # Check if pagination is needed (GitHub API limits to 100 items per page)
        if 'Link' in response.headers and 'next' in response.headers['Link']:
            print("More than 100 repositories found. Implementing pagination...")
            # Implement pagination logic here if needed

        # Filter out forks if not wanted
        if not include_forks:
            repos = [repo for repo in repos if not repo['fork']]

        # Select relevant columns for the DataFrame
        repo_data = []
        for repo in repos:
            repo_data.append(repo)

        # Create DataFrame
        df = pd.DataFrame(repo_data)
        print(f"Created DataFrame with {len(df)} repositories")
        return df
    else:
        print(f"Failed to fetch repositories: {response.status_code}")
        print(response.json())
        return None

In [78]:
user = test_api_connection()

Successfully connected as: Dappner


In [79]:
repos_df = get_my_repositories()

More than 100 repositories found. Implementing pagination...
Created DataFrame with 99 repositories


In [80]:
repos_df

,id,node_id,name,full_name,private,owner,html_url,description,fork,url,...,allow_forking,is_template,web_commit_signoff_required,topics,visibility,forks,open_issues,watchers,default_branch,permissions
0,706274562,R_kgDOKhjlAg,admin,arch-intelligence/admin,True,"{'login': 'arch-intelligence', 'id': 134008031...",https://github.com/arch-intelligence/admin,None,False,https://api.github.com/repos/arch-intelligence...,...,False,False,False,[],private,0,0,0,master,"{'admin': False, 'maintain': False, 'push': Fa..."
1,781097993,R_kgDOLo6cCQ,ai.arch,arch-intelligence/ai.arch,True,"{'login': 'arch-intelligence', 'id': 134008031...",https://github.com/arch-intelligence/ai.arch,Arch static site powered by Next.js & Sanity,False,https://api.github.com/repos/arch-intelligence...,...,False,False,False,[],private,0,0,0,main,"{'admin': False, 'maintain': False, 'push': Fa..."
2,889997438,R_kgDONQxIfg,app.arch.cdn,arch-intelligence/app.arch.cdn,True,"{'login': 'arch-intelligence', 'id': 134008031...",https://github.com/arch-intelligence/app.arch.cdn,Content Delivery for Arch,False,https://api.github.com/repos/arch-intelligence...,...,False,False,False,[],private,0,0,0,prod,"{'admin': False, 'maintain': False, 'push': Fa..."
3,842161246,R_kgDOMjJcXg,app.arch.platform.oblivion,arch-intelligence/app.arch.platform.oblivion,True,"{'login': 'arch-intelligence', 'id': 134008031...",https://github.com/arch-intelligence/app.arch....,Rust backend for Leasing Platform,False,https://api.github.com/repos/arch-intelligence...,...,False,False,False,[],private,0,0,0,prod,"{'admin': False, 'maintain': False, 'push': Fa..."
4,755611948,R_kgDOLQm5LA,arch,arch-intelligence/arch,True,"{'login': 'arch-intelligence', 'id': 134008031...",https://github.com/arch-intelligence/arch,Full mono-repo of the Arch.ai project & all it...,False,https://api.github.com/repos/arch-intelligence...,...,False,False,False,[],private,0,0,0,main,"{'admin': False, 'maintain': False, 'push': Fa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,521835593,R_kgDOHxqUSQ,ao-website,LeadReel/ao-website,True,"{'login': 'LeadReel', 'id': 97260566, 'node_id...",https://github.com/LeadReel/ao-website,None,False,https://api.github.com/repos/LeadReel/ao-website,...,False,False,False,[],private,0,0,0,main,"{'admin': True, 'maintain': True, 'push': True..."
95,546871922,R_kgDOIJiacg,application-portal-api,LeadReel/application-portal-api,True,"{'login': 'LeadReel', 'id': 97260566, 'node_id...",https://github.com/LeadReel/application-portal...,The API for the Application Portal,False,https://api.github.com/repos/LeadReel/applicat...,...,False,False,False,[],private,0,0,0,master,"{'admin': True, 'maintain': True, 'push': True..."
96,453432193,R_kgDOGwbTgQ,application-portal-master,LeadReel/application-portal-master,True,"{'login': 'LeadReel', 'id': 97260566, 'node_id...",https://github.com/LeadReel/application-portal...,Application Portal Master template,False,https://api.github.com/repos/LeadReel/applicat...,...,False,False,False,[],private,0,0,0,master,"{'admin': True, 'maintain': True, 'push': True..."
97,453433947,R_kgDOGwbaWw,application-portal-payments,LeadReel/application-portal-payments,True,"{'login': 'LeadReel', 'id': 97260566, 'node_id...",https://github.com/LeadReel/application-portal...,The payments backend for the application porta...,False,https://api.github.com/repos/LeadReel/applicat...,...,False,False,False,[],private,0,0,0,master,"{'admin': True, 'maintain': True, 'push': True..."


In [81]:
my_repos = repos_df[repos_df["full_name"].str.contains("Dappner/") ]

In [85]:
my_repos

,id,node_id,name,full_name,private,owner,html_url,description,fork,url,...,allow_forking,is_template,web_commit_signoff_required,topics,visibility,forks,open_issues,watchers,default_branch,permissions
43,516494739,R_kgDOHskVkw,auth0-api,Dappner/auth0-api,True,"{'login': 'Dappner', 'id': 44614306, 'node_id'...",https://github.com/Dappner/auth0-api,None,False,https://api.github.com/repos/Dappner/auth0-api,...,True,False,False,[],private,0,0,0,master,"{'admin': True, 'maintain': True, 'push': True..."
44,922358152,R_kgDONvoRiA,c24-probeaufgabe,Dappner/c24-probeaufgabe,True,"{'login': 'Dappner', 'id': 44614306, 'node_id'...",https://github.com/Dappner/c24-probeaufgabe,None,False,https://api.github.com/repos/Dappner/c24-probe...,...,True,False,False,[],private,0,0,0,main,"{'admin': True, 'maintain': True, 'push': True..."
45,484477091,R_kgDOHOCIow,colibri-control,Dappner/colibri-control,True,"{'login': 'Dappner', 'id': 44614306, 'node_id'...",https://github.com/Dappner/colibri-control,None,False,https://api.github.com/repos/Dappner/colibri-c...,...,True,False,False,[],private,0,0,0,main,"{'admin': True, 'maintain': True, 'push': True..."
46,445949140,R_kgDOGpSk1A,colibri-dashboard,Dappner/colibri-dashboard,True,"{'login': 'Dappner', 'id': 44614306, 'node_id'...",https://github.com/Dappner/colibri-dashboard,None,False,https://api.github.com/repos/Dappner/colibri-d...,...,True,False,False,[],private,0,0,0,main,"{'admin': True, 'maintain': True, 'push': True..."
47,536610601,R_kgDOH_wHKQ,colibri-service-metric-aggregation-api,Dappner/colibri-service-metric-aggregation-api,True,"{'login': 'Dappner', 'id': 44614306, 'node_id'...",https://github.com/Dappner/colibri-service-met...,None,False,https://api.github.com/repos/Dappner/colibri-s...,...,True,False,False,[],private,0,0,0,master,"{'admin': True, 'maintain': True, 'push': True..."
48,420529986,R_kgDOGRDHQg,Dappner,Dappner/Dappner,False,"{'login': 'Dappner', 'id': 44614306, 'node_id'...",https://github.com/Dappner/Dappner,None,False,https://api.github.com/repos/Dappner/Dappner,...,True,False,False,[],public,0,0,0,main,"{'admin': True, 'maintain': True, 'push': True..."
49,420274423,R_kgDOGQzg9w,Dash-Stock,Dappner/Dash-Stock,True,"{'login': 'Dappner', 'id': 44614306, 'node_id'...",https://github.com/Dappner/Dash-Stock,A Stock Web Interface,False,https://api.github.com/repos/Dappner/Dash-Stock,...,True,False,False,[],private,0,0,0,main,"{'admin': True, 'maintain': True, 'push': True..."
50,929896835,R_kgDON20Zgw,dotfiles,Dappner/dotfiles,True,"{'login': 'Dappner', 'id': 44614306, 'node_id'...",https://github.com/Dappner/dotfiles,None,False,https://api.github.com/repos/Dappner/dotfiles,...,True,False,False,[],private,0,0,0,main,"{'admin': True, 'maintain': True, 'push': True..."
51,951428745,R_kgDOOLWmiQ,ecse211-final,Dappner/ecse211-final,False,"{'login': 'Dappner', 'id': 44614306, 'node_id'...",https://github.com/Dappner/ecse211-final,None,False,https://api.github.com/repos/Dappner/ecse211-f...,...,True,False,False,[],public,0,0,0,main,"{'admin': True, 'maintain': True, 'push': True..."
52,928512991,R_kgDON1f73w,ECSE211_proj_1,Dappner/ECSE211_proj_1,True,"{'login': 'Dappner', 'id': 44614306, 'node_id'...",https://github.com/Dappner/ECSE211_proj_1,None,False,https://api.github.com/repos/Dappner/ECSE211_p...,...,True,False,False,[],private,0,0,0,main,"{'admin': True, 'maintain': True, 'push': True..."


In [112]:
commits_url = "https://api.github.com/repos/Dappner/my-dashboard/commits?author=Dappner"
events_url ="https://api.github.com/repos/Dappner/my-dashboard/events"
commits_res = session.get(commits_url, params={'per_page': 100})
events_res = session.get(events_url, params={'per_page': 100})

In [114]:
pd.DataFrame(events_res.json())

,id,type,actor,repo,payload,public,created_at
0,48811562084,PushEvent,"{'id': 44614306, 'login': 'Dappner', 'display_...","{'id': 942249447, 'name': 'Dappner/my-dashboar...","{'repository_id': 942249447, 'push_id': 238200...",False,2025-04-17T22:20:48Z
1,48810179270,PushEvent,"{'id': 44614306, 'login': 'Dappner', 'display_...","{'id': 942249447, 'name': 'Dappner/my-dashboar...","{'repository_id': 942249447, 'push_id': 238192...",False,2025-04-17T21:15:45Z
2,48809622376,PushEvent,"{'id': 44614306, 'login': 'Dappner', 'display_...","{'id': 942249447, 'name': 'Dappner/my-dashboar...","{'repository_id': 942249447, 'push_id': 238189...",False,2025-04-17T20:54:05Z
3,48809393159,PushEvent,"{'id': 44614306, 'login': 'Dappner', 'display_...","{'id': 942249447, 'name': 'Dappner/my-dashboar...","{'repository_id': 942249447, 'push_id': 238188...",False,2025-04-17T20:45:11Z
4,48805995365,PushEvent,"{'id': 44614306, 'login': 'Dappner', 'display_...","{'id': 942249447, 'name': 'Dappner/my-dashboar...","{'repository_id': 942249447, 'push_id': 238171...",False,2025-04-17T18:44:58Z
...,...,...,...,...,...,...,...
95,48115960846,PushEvent,"{'id': 44614306, 'login': 'Dappner', 'display_...","{'id': 942249447, 'name': 'Dappner/my-dashboar...","{'repository_id': 942249447, 'push_id': 234599...",False,2025-03-30T03:02:45Z
96,48115627682,PushEvent,"{'id': 44614306, 'login': 'Dappner', 'display_...","{'id': 942249447, 'name': 'Dappner/my-dashboar...","{'repository_id': 942249447, 'push_id': 234596...",False,2025-03-30T02:23:26Z
97,48115577407,PushEvent,"{'id': 44614306, 'login': 'Dappner', 'display_...","{'id': 942249447, 'name': 'Dappner/my-dashboar...","{'repository_id': 942249447, 'push_id': 234596...",False,2025-03-30T02:17:16Z
98,48115301284,PushEvent,"{'id': 44614306, 'login': 'Dappner', 'display_...","{'id': 942249447, 'name': 'Dappner/my-dashboar...","{'repository_id': 942249447, 'push_id': 234594...",False,2025-03-30T01:45:11Z


In [101]:
commits_df = pd.DataFrame(commits_res.json())

In [102]:
commits_df

,sha,node_id,commit,url,html_url,comments_url,author,committer,parents
0,bff769d3016bdbfad5641ffa2588c51805750406,C_kwDOOCmV59oAKGJmZjc2OWQzMDE2YmRiZmFkNTY0MWZm...,"{'author': {'name': 'Dappner', 'email': 'nickl...",https://api.github.com/repos/Dappner/my-dashbo...,https://github.com/Dappner/my-dashboard/commit...,https://api.github.com/repos/Dappner/my-dashbo...,"{'login': 'Dappner', 'id': 44614306, 'node_id'...","{'login': 'Dappner', 'id': 44614306, 'node_id'...",[{'sha': 'f2b05e51d52ec30a0c84dfe180b950f99934...
1,f2b05e51d52ec30a0c84dfe180b950f999345a11,C_kwDOOCmV59oAKGYyYjA1ZTUxZDUyZWMzMGEwYzg0ZGZl...,"{'author': {'name': 'Dappner', 'email': 'nickl...",https://api.github.com/repos/Dappner/my-dashbo...,https://github.com/Dappner/my-dashboard/commit...,https://api.github.com/repos/Dappner/my-dashbo...,"{'login': 'Dappner', 'id': 44614306, 'node_id'...","{'login': 'Dappner', 'id': 44614306, 'node_id'...",[{'sha': '031bc78dd65ad5c42807416e0c042c6f8b7b...
2,031bc78dd65ad5c42807416e0c042c6f8b7bd62e,C_kwDOOCmV59oAKDAzMWJjNzhkZDY1YWQ1YzQyODA3NDE2...,"{'author': {'name': 'Dappner', 'email': 'nickl...",https://api.github.com/repos/Dappner/my-dashbo...,https://github.com/Dappner/my-dashboard/commit...,https://api.github.com/repos/Dappner/my-dashbo...,"{'login': 'Dappner', 'id': 44614306, 'node_id'...","{'login': 'Dappner', 'id': 44614306, 'node_id'...",[{'sha': 'ad13e4eadebcc3fb3dd0414f5944ef267dc2...
3,ad13e4eadebcc3fb3dd0414f5944ef267dc2f7c3,C_kwDOOCmV59oAKGFkMTNlNGVhZGViY2MzZmIzZGQwNDE0...,"{'author': {'name': 'Dappner', 'email': 'nickl...",https://api.github.com/repos/Dappner/my-dashbo...,https://github.com/Dappner/my-dashboard/commit...,https://api.github.com/repos/Dappner/my-dashbo...,"{'login': 'Dappner', 'id': 44614306, 'node_id'...","{'login': 'Dappner', 'id': 44614306, 'node_id'...",[{'sha': '71a128d17ab415c1ff3ef5f7ce7ba81c24be...
4,71a128d17ab415c1ff3ef5f7ce7ba81c24be43d5,C_kwDOOCmV59oAKDcxYTEyOGQxN2FiNDE1YzFmZjNlZjVm...,"{'author': {'name': 'Dappner', 'email': 'nickl...",https://api.github.com/repos/Dappner/my-dashbo...,https://github.com/Dappner/my-dashboard/commit...,https://api.github.com/repos/Dappner/my-dashbo...,"{'login': 'Dappner', 'id': 44614306, 'node_id'...","{'login': 'Dappner', 'id': 44614306, 'node_id'...",[{'sha': '75cb057961f07807f6b68afddc8233378827...
...,...,...,...,...,...,...,...,...,...
95,0e08c1fb0f8b30fb6ae5989318e696d30970829b,C_kwDOOCmV59oAKDBlMDhjMWZiMGY4YjMwZmI2YWU1OTg5...,"{'author': {'name': 'Dappner', 'email': 'nickl...",https://api.github.com/repos/Dappner/my-dashbo...,https://github.com/Dappner/my-dashboard/commit...,https://api.github.com/repos/Dappner/my-dashbo...,"{'login': 'Dappner', 'id': 44614306, 'node_id'...","{'login': 'Dappner', 'id': 44614306, 'node_id'...",[{'sha': '234ed450b6ce1978dff81fed18ba428bba57...
96,234ed450b6ce1978dff81fed18ba428bba579ec5,C_kwDOOCmV59oAKDIzNGVkNDUwYjZjZTE5NzhkZmY4MWZl...,"{'author': {'name': 'Dappner', 'email': 'nickl...",https://api.github.com/repos/Dappner/my-dashbo...,https://github.com/Dappner/my-dashboard/commit...,https://api.github.com/repos/Dappner/my-dashbo...,"{'login': 'Dappner', 'id': 44614306, 'node_id'...","{'login': 'Dappner', 'id': 44614306, 'node_id'...",[{'sha': 'a28091c1b8c6a298bd679ada60c6f5b8eea3...
97,a28091c1b8c6a298bd679ada60c6f5b8eea3b3d5,C_kwDOOCmV59oAKGEyODA5MWMxYjhjNmEyOThiZDY3OWFk...,"{'author': {'name': 'Dappner', 'email': 'nickl...",https://api.github.com/repos/Dappner/my-dashbo...,https://github.com/Dappner/my-dashboard/commit...,https://api.github.com/repos/Dappner/my-dashbo...,"{'login': 'Dappner', 'id': 44614306, 'node_id'...","{'login': 'Dappner', 'id': 44614306, 'node_id'...",[{'sha': '53f5c6251a74202d5f8de3592539b27d2af4...
98,53f5c6251a74202d5f8de3592539b27d2af4bb82,C_kwDOOCmV59oAKDUzZjVjNjI1MWE3NDIwMmQ1ZjhkZTM1...,"{'author': {'name': 'Dappner', 'email': 'nickl...",https://api.github.com/repos/Dappner/my-dashbo...,https://github.com/Dappner/my-dashboard/commit...,https://api.github.com/repos/Dappner/my-dashbo..

In [104]:
one_commit = commits_df.iloc[0]

In [106]:
one_commit["commit"]

{'author': {'name': 'Dappner',
  'email': 'nicklasastorian@googlemail.com',
  'date': '2025-04-17T22:20:44Z'},
 'committer': {'name': 'Dappner',
  'email': 'nicklasastorian@googlemail.com',
  'date': '2025-04-17T22:20:44Z'},
 'message': 'more',
 'tree': {'sha': '842400682a21cf0464037e0c65d715cc08b8f270',
  'url': 'https://api.github.com/repos/Dappner/my-dashboard/git/trees/842400682a21cf0464037e0c65d715cc08b8f270'},
 'url': 'https://api.github.com/repos/Dappner/my-dashboard/git/commits/bff769d3016bdbfad5641ffa2588c51805750406',
 'comment_count': 0,
 'verification': {'verified': False,
  'reason': 'unsigned',
  'signature': None,
  'payload': None,
  'verified_at': None}}

In [108]:
data_from_commit = session.get("https://api.github.com/repos/Dappner/my-dashboard/commits/bff769d3016bdbfad5641ffa2588c51805750406")

In [111]:
data_from_commit.json()

{'sha': 'bff769d3016bdbfad5641ffa2588c51805750406',
 'node_id': 'C_kwDOOCmV59oAKGJmZjc2OWQzMDE2YmRiZmFkNTY0MWZmYTI1ODhjNTE4MDU3NTA0MDY',
 'commit': {'author': {'name': 'Dappner',
   'email': 'nicklasastorian@googlemail.com',
   'date': '2025-04-17T22:20:44Z'},
  'committer': {'name': 'Dappner',
   'email': 'nicklasastorian@googlemail.com',
   'date': '2025-04-17T22:20:44Z'},
  'message': 'more',
  'tree': {'sha': '842400682a21cf0464037e0c65d715cc08b8f270',
   'url': 'https://api.github.com/repos/Dappner/my-dashboard/git/trees/842400682a21cf0464037e0c65d715cc08b8f270'},
  'url': 'https://api.github.com/repos/Dappner/my-dashboard/git/commits/bff769d3016bdbfad5641ffa2588c51805750406',
  'comment_count': 0,
  'verification': {'verified': False,
   'reason': 'unsigned',
   'signature': None,
   'payload': None,
   'verified_at': None}},
 'url': 'https://api.github.com/repos/Dappner/my-dashboard/commits/bff769d3016bdbfad5641ffa2588c51805750406',
 'html_url': 'https://github.com/Dappner/my-da